<a href="https://colab.research.google.com/github/deiteris/voice-changer/blob/master-custom/Colab_RealtimeVoiceChanger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [w-okada's Voice Changer](https://github.com/deiteris/voice-changer) | **Google Colab**

---

## **READ ME - VERY IMPORTANT**

You can use the following settings for optimal results:

Best performance (with good quality): `f0: fcpe | Chunk: 80.0ms or higher | Extra: 5s`<br>
Best quality: `f0: rmvpe | Chunk: 112.0ms or higher | Extra: 5s`<br>
**Don't forget to select your Colab GPU in the GPU field (<b>Tesla T4</b>, for free users)*

You can tune `Chunk` for lower/higher delay and `Extra` for better quality.

<br><br>

---

### Always use Colab GPU (**VERY VERY VERY IMPORTANT!**)
You need to use a Colab GPU so the Voice Changer can work faster and better\
Use the menu above and click on **Runtime** » **Change runtime** » **Hardware acceleration** to select a GPU (**T4 is the free one**)

---

# **Credits and Support**
Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Original instructions by [Hina](https://github.com/HinaBl)<br>

Need help? [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

---

In [ ]:
#=================Updated=================
# @title **[1]** Clone repository and install dependencies
# @markdown This first step will download the latest version of Voice Changer and install the dependencies. **It can take some time to complete.**
import os
import time
from google.colab import drive

#@markdown ---
# @title **[Optional]** Connect to Google Drive
# @markdown Using Google Drive will automatically save your uploaded models for later use. Make sure you have sufficient amount of space on your Google Drive.

%cd /content

Use_Drive=True #@param {type:"boolean"}

!pip install colorama python-dotenv pyngrok --quiet

from colorama import Fore, Style
import requests

print(f"{Fore.CYAN}> Downloading prebuilt executable...{Style.RESET_ALL}")

res = requests.get('https://api.github.com/repos/deiteris/voice-changer/releases/latest')
release_info = res.json()

for asset in release_info['assets']:
   if not asset['name'].startswith('voice-changer-linux-amd64-cuda.tar.gz'):
      continue
   download_url = asset['browser_download_url']
   !wget -q --show-progress {download_url}

print(f"{Fore.GREEN}> Unpacking...{Style.RESET_ALL}")
!cat voice-changer-linux-amd64-cuda.tar.gz.* | tar xzf -
print(f"{Fore.GREEN}> Finished unpacking!{Style.RESET_ALL}")
!rm -rf voice-changer-linux-amd64-cuda.tar.gz.*

%cd MMVCServerSIO

if Use_Drive and not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

  !mkdir -p /content/drive/MyDrive/voice-changer/server/model_dir
  !mkdir -p /content/drive/MyDrive/voice-changer/server/pretrain
  !rm -rf /content/MMVCServerSIO/server/model_dir
  !rm -rf /content/MMVCServerSIO/server/pretrain

  time.sleep(5)

  os.symlink("/content/drive/MyDrive/voice-changer/server/model_dir", "/content/MMVCServerSIO/model_dir", True)
  os.symlink("/content/drive/MyDrive/voice-changer/server/pretrain", "/content/MMVCServerSIO/pretrain", True)

print(f"{Fore.GREEN}> Successfully downloaded and unpacked the binary!{Style.RESET_ALL}")

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!apt-get -y install libportaudio2 -qq

In [ ]:
#=================Updated=================
# @title **[2]** Set server configuration
# @markdown This cell will set the server configuration.

%cd /content/MMVCServerSIO

from dotenv import set_key

set_key('.env', "SAMPLE_MODE", "")

Ready = True

In [ ]:

    #@title **[Optional Cell For V1 Old Okada Voice Changer]** Upload a voice model (Run this before running the Voice Changer)
    #@markdown Find your model here [voice-models](https://voice-models.com/)
    # @markdown ---
    model_slot = "0" #@param ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199']

    !rm -rf model_dir/$model_slot
    #@markdown **[Optional]** Add an icon to the model (Kosongin aja gapapa / It's okay to leave it blank)
    icon_link = ""  #@param {type:"string"}
    icon_link = '"'+icon_link+'"'
    !mkdir model_dir
    !mkdir model_dir/$model_slot
    #@markdown Put your model's download link here `(must be a zip file and don't use GPT-SoVITS Model)` only supports **huggingface.co** & **google drive**<br>
    model_link = "https://huggingface.co/makiligon/RVC-Models/resolve/main/Furina.zip"  #@param {type:"string"}

    if model_link.startswith("https://www.weights.gg") or model_link.startswith("https://weights.gg"):
        print("Links from weights.gg is no longer supported.")
        sys.exit()
    elif model_link.startswith("https://drive.google.com"):
        model_link = '"'+model_link+'"'
        !gdown $model_link --fuzzy -O model.zip
        print("Model from Drive")
    elif model_link.startswith("https://huggingface.co"):
        model_link = model_link
        model_link = '"'+model_link+'"'
        !curl -L $model_link > model.zip
        print("Model from huggingface Link")
    else:
        model_link = model_link
        model_link = '"'+model_link+'"'
        !curl -L -O $model_link
        !mv ./*.pth model_dir/$model_slot/
        print('Model(.pth) or a direct model link.')

    # Conditionally set the iconFile based on whether icon_link is empty
    if icon_link == '""':
        iconFile = ""
        print("icon_link is empty, so no icon file will be downloaded.")
    else:
        iconFile = "icon.png"
        !curl -L $icon_link > model_dir/$model_slot/icon.png

    !unzip model.zip -d model_dir/$model_slot

    !mv model_dir/$model_slot/*/* model_dir/$model_slot/
    !rm -rf model_dir/$model_slot/*/
    !rm -rf model.zip
    #@markdown **Model Voice Conversion Setting**
    Tune = 12  #@param {type:"slider",min:-24,max:24,step:1}
    Index = 0  #@param {type:"slider",min:0,max:1,step:0.1}

    param_link = ""
    if param_link == "":
        paramset = requests.get("https://pastebin.com/raw/P6ar4baa").text
        exec(paramset)

    clear_output()
    print("\033[93mModel with the name of "+model_name+" has been Imported to slot "+model_slot)


In [ ]:
#=======================Updated=========================

# @title Start Server **using ngrok**
# @markdown This cell will start the server, the first time that you run it will download the models, so it can take a few minutes (usually ~1-2 minutes)

# @markdown ---
# @markdown You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
# @markdown ---
# @markdown **1** - Create a <font color=green>**free**</font> account at [ngrok](https://dashboard.ngrok.com/signup) or **login with Google/Github account**\
# @markdown **2** - If you didn't logged in with Google/Github, you will need to **verify your e-mail**!\
# @markdown **3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and place it here:
Token = 'TOKEN_HERE' # @param {type:"string"}
# @markdown **4** - *(optional)* Change to a region near to you\
# @markdown `Default Region: ap - Asia/Pacific (Singapore)`
Region = "ap - Asia/Pacific (Singapore)" # @param ["ap - Asia/Pacific (Singapore)", "au - Australia (Sydney)","eu - Europe (Frankfurt)", "in - India (Mumbai)","jp - Japan (Tokyo)","sa - South America (Sao Paulo)", "us - United States (Ohio)"]

#@markdown **5** - *(optional)* Other options:
ClearConsole = True  # @param {type:"boolean"}

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!
# ---------------------------------

if not globals().get('Ready', False):
    print("Go back and run first and second cells.")
else:
    from pyngrok import conf, ngrok
    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    conf.get_default().authtoken = Token
    conf.get_default().region = Region
    conf.set_default(MyConfig);

    import threading, time, socket
    PORT = 18888

    import json
    from pyngrok import ngrok
    from IPython.display import clear_output

    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url
    set_key('.env', "ALLOWED_ORIGINS", json.dumps([public_url]))

    def wait_for_server():
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', PORT))
            if result == 0:
                break
            sock.close()
        if ClearConsole:
            clear_output()
        print("--------- SERVER READY! ---------")
        print("Your server is available at:")
        print(public_url)
        print("---------------------------------")

    threading.Thread(target=wait_for_server, daemon=True).start()

    !./MMVCServerSIO

    ngrok.disconnect(ngrokConnection.public_url)